In [1]:
import numpy as np
import xarray as xr
import pandas as pd 
import sys 
from EchoPro import EchoPro
epro_2019 = EchoPro(init_file_path='./config_files/initialization_config.yml',
                    survey_year_file_path='./config_files/survey_year_2019_config.yml',
                    source=3,
                    bio_data_type=1,
                    age_data_status=1)

A check of the initialization file needs to be done!
A check of the survey year file needs to be done!
Loading US biological data ...


In [2]:
%%time
# code to turn length_US file into a dataframe with expanded frequencies
out = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_length_US'])
df = epro_2019.process_length_data_df(out, 0)
df.head()

CPU times: user 234 ms, sys: 1.61 ms, total: 236 ms
Wall time: 235 ms


,Sex,Length,Male_ind,Female_ind,n,meanlen,stdlen,TS_lin,TS_sd,TS_log,nM,nF
Haul,,,,,,,,,,,,
1,"[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","[20.0, 20.0, 20.0, 20.0, 20.0, 21.0, 21.0, 21....","[148, 149, 150, 151, 152, 153, 154, 155, 156, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",342,23.385965,2.125845,-40.585259,0.747452,-40.654102,194,148
3,"[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","[18.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19....","[128, 129, 130, 131, 132, 133, 134, 135, 136, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",535,20.949533,1.400667,-41.557179,0.704258,-41.600977,97,128
7,"[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","[20.0, 20.0, 20.0, 20.0, 21.0, 21.0, 21.0, 21....","[72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 8...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",167,22.239521,1.247695,-41.043925,0.489027,-41.071140,95,72
8,"[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","[20.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21....","[68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 7...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",161,23.273292,1.781809,-40.637619,0.638929,-40.686783,93,68
9,"[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","[20.0, 20.0, 20.0, 20.0, 21.0, 21.0, 21.0, 21....","[63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 7...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",127,22.803150,3.023819,-40.764991,0.883306,-40.892414,64,63


In [106]:
df = epro_2019.process_length_data_ds(out, 0)
df.set_index('Haul', inplace=True)
print(df.head())
max_haul_size = df.index.value_counts().max()
series_haul = df.groupby('Haul').apply(np.array)

      Sex  Length  Frequency
Haul                        
1       2      20          5
1       2      21         13
1       2      22         38
1       2      23         38
1       2      24         16


In [108]:
# pad the numpy arrays of each haul to match the maximum haul size
series_haul_pad = series_haul.apply(lambda x: np.vstack((x, np.nan*np.ones((max_haul_size - x.shape[0], 3)))) if x.shape[0] != max_haul_size else x)

In [110]:
# series_haul_pad = series_haul_pad.apply(np.transpose)

In [120]:
# np_haul_pad = series_haul_pad.agg(lambda x: np.hstack(x.values))

In [125]:
# np_haul_pad.shape

In [124]:
# ds = xr.Dataset(data_vars={
#     'Sex': (['Haul'], np_haul_pad[0, :]),
#     'Length': (['Haul'], np_haul_pad[1, :]), 
#     'Frequency': (['Haul'], np_haul_pad[2, :])}, coords={'Haul': df.index})

In [4]:
# sys.getsizeof(df)

In [5]:
# %%time
# max_n = df['n'].max()
# sex_series = df.Sex.apply(lambda x: np.concatenate([x, np.nan*np.ones(max_n - len(x))])  if len(x) != max_n else x)
# sex_stacked = np.vstack(sex_series)

# length_series = df.Length.apply(lambda x: np.concatenate([x, np.nan*np.ones(max_n - len(x))])  if len(x) != max_n else x)
# length_stacked = np.vstack(length_series)

# male_ind_series = df.Male_ind.apply(lambda x: np.concatenate([x, np.nan*np.ones(max_n - len(x))])  if len(x) != max_n else x)
# male_ind_stacked = np.vstack(male_ind_series)

# female_ind_series = df.Female_ind.apply(lambda x: np.concatenate([x, np.nan*np.ones(max_n - len(x))])  if len(x) != max_n else x)
# female_ind_stacked = np.vstack(female_ind_series)

In [6]:
# test_dataset = xr.Dataset(dict(
#     Sex=(['haul', 'index'], sex_stacked), 
#     Length=(['haul', 'index'], length_stacked),
#     Male_ind=(['haul', 'index'], male_ind_stacked),
#     Female_ind=(['haul', 'index'], female_ind_stacked), 
#     n=(['haul'], df.n.to_numpy()),
#     meanlen=(['haul'], df.meanlen.to_numpy()),
#     stdlen=(['haul'], df.stdlen.to_numpy()),
#     TS_lin=(['haul'], df.TS_lin.to_numpy()),
#     TS_sd=(['haul'], df.TS_sd.to_numpy()),
#     TS_log=(['haul'], df.TS_log.to_numpy()),
#     nM=(['haul'], df.nM.to_numpy()),
#     nF=(['haul'], df.nF.to_numpy())
# ), coords={
#     'haul': df.index.to_numpy(),
#     'index' : range(max_n)
# })

In [7]:
# test_dataset

In [8]:
# test_dataset.nbytes

In [9]:
# %%time
# out = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_catch_US'])
# df = epro_2019.process_catch_data(out)

In [10]:
# %%time
# out = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_specimen_US'])
# df = epro_2019.process_specimen_data(out, 0)
# df.head()

In [11]:
# catch_us_df, length_us_df, trawl_us_df, gear_us_df, specimen_us_df = epro_2019.load_biological_data_us()
# catch_can_df, length_can_df, trawl_can_df, gear_can_df, specimen_can_df = epro_2019.load_biological_data_canada()

In [12]:
# %%time
# out = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_length_US'])
# df = epro_2019.process_length_data(out, 0)
# df.head()

In [13]:
# epro_2019.specimen_df

In [14]:
# specimen_ds = epro_2019.specimen_df.to_xarray()

In [15]:

# select the indices that do not have nan in either Length or Weight
# len_wgt_nonull = np.logical_and(specimen_ds.Length.notnull(), specimen_ds.Weight.notnull())

# x = np.log10(specimen_ds.Length.sel(Haul=len_wgt_nonull)).values
# y = np.log10(specimen_ds.Weight.sel(Haul=len_wgt_nonull)).values

# p = np.polyfit(x, y, 1)  # linear regression 

# print(y[np.isnan(y)])

# print(x.shape)
# print(y.shape)

# print(p)

In [16]:
# specimen_ds.sel(Haul=1)